In [1]:
%logstop
%logstart -ortq ~/.logs/2020-05-20_ML_office-hours_23.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

Logging hadn't been started.


/Users/nicholascifuentes-goodbody/anaconda3/envs/wq3/lib/python3.7/site-packages/IPython/core/magics/logging.py:130: UserWarning: Couldn't start log: [Errno 2] No such file or directory: '/Users/nicholascifuentes-goodbody/.logs/2020-05-20_ML_office-hours_23.py'
  warn("Couldn't start log: %s" % sys.exc_info()[1])


# Import and EDA

In [2]:
import pandas as pd

In [3]:
X = pd.read_csv('data/training-set-values.csv', index_col=[0])

In [ ]:
X.head()

In [4]:
y = pd.read_csv('data/training-set-labels.csv', index_col=[0])
y = y.status_group
y.head()

id
69572        functional
8776         functional
34310        functional
67743    non functional
19728        functional
Name: status_group, dtype: object

In [ ]:
y.status_group.value_counts()

# Make MVP: Model with One Feature

In [5]:
# Train-test split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
X = X[['population']]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,
                                               test_size=0.2,
                                               random_state=42)

In [ ]:
model_mvp = LogisticRegression(solver='lbfgs', multi_class='auto')
model_mvp.fit(X_train, y_train);

In [ ]:
print('in-sample accuracy:', model_mvp.score(X_train, y_train))
print('out-sample accuracy:', model_mvp.score(X_test, y_test))

# Prepare Our First Submission

In [29]:
X_validation = pd.read_csv('data/test-set-values.csv',
                           index_col=[0])

In [ ]:
X_validation = X_validation[['population']]

In [ ]:
X_validation.head()

In [ ]:
y_pred = model_mvp.predict(X_validation)

In [ ]:
submission = pd.DataFrame(y_pred, index=X_validation.index, 
                          columns=['status_group'])

In [ ]:
submission.to_csv('submissions/model_mvp.csv')

In [30]:
# Let's make a function to simplify submission
def make_submission(model, X_validation):
    y_pred = model.predict(X_validation)
    submission = pd.DataFrame(y_pred, 
                              index=X_validation.index, 
                              columns=['status_group'])
    time_stamp = pd.Timestamp.now().strftime(format='%Y-%m-%d_%H-%M_')
    submission.to_csv(f'submissions/{time_stamp}submission.csv')

In [ ]:
make_submission(model_mvp, X_validation)

# Make model with all numerical features

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = X.select_dtypes('number')

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,
                                               test_size=0.2,
                                               random_state=42)

In [ ]:
model_numeric = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=100))
])

In [ ]:
#model_numeric.fit(X_train, y_train);

In [ ]:
#print('in-sample accuracy:', model_numeric.score(X_train, y_train))
#print('out-sample accuracy:', model_numeric.score(X_test, y_test))

In [ ]:
#X_validation = X_validation.select_dtypes('number')

In [ ]:
#make_submission(model_numeric, X_validation)

# Make Model with All Features

In [20]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [32]:
num_features = X.select_dtypes('number').columns.tolist()
cat_features = X.select_dtypes('object').columns.tolist()

In [34]:
cat_features.pop(0)

'date_recorded'

In [35]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

ct = ColumnTransformer([
    ('numerical', num_pipe, num_features),
    ('categorical', cat_pipe, cat_features)
])

model_all_feat = Pipeline([
    ('transformer', ct),
    ('classifier', RandomForestClassifier(n_estimators=20,
                                          max_features='sqrt',
                                          n_jobs=10))
])

In [36]:
model_all_feat.fit(X,y);

Pipeline(memory=None,
         steps=[('transformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                          

In [37]:
model_all_feat.score(X,y)

0.9955555555555555

In [38]:
make_submission(model_all_feat, X_validation)

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*